# Compare to Other Models

- Robert Shaw
- Data Driven March Madness

In this file, we test code written in march_madness_classes. The code allows us to compare the results of different brackets in order to see which one is better, or to see the similiarity between the two.

In [1]:
import march_madness_classes as mmc
import march_madness_games as mmg
import march_madness_models as mmm
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression as LogReg

In [2]:
# read in the dataset
seeds = pd.read_csv("datasets/kaggle_data/TourneySeeds.csv")
slots = pd.read_csv("datasets/kaggle_data/TourneySlots.csv")
games = pd.read_csv("datasets/kaggle_data/TourneyCompactResults.csv")

seeds_arr = mmg.filter_into_seasons(seeds)
slots_arr = mmg.filter_into_seasons(slots)
games_arr = mmg.filter_into_seasons(games)

In [11]:
# extract predictors
markov_data = pd.read_csv("datasets/our_data/stationary", index_col=0)
consistency = pd.read_csv("datasets/our_data/consistency", index_col=0)
dominance = pd.read_csv("datasets/our_data/dominance", index_col=0)
past_resul = pd.read_csv("datasets/our_data/past_results", index_col=0)
rpi = pd.read_csv("datasets/our_data/rpi", index_col=0)
bad_losses = pd.read_csv("datasets/our_data/bad_losses", index_col=0)
tough_wins = pd.read_csv("datasets/our_data/tough_wins", index_col=0)
close_games= pd.read_csv("datasets/our_data/close_games", index_col=0)
close_wins = pd.read_csv("datasets/our_data/close_wins",index_col=0)
close_wins_perc = pd.read_csv("datasets/our_data/close_wins_perc", index_col=0)
momentum = pd.read_csv("datasets/our_data/momentum", index_col=0)
weighted_wins = pd.read_csv("datasets/our_data/weighted_wins", index_col=0)
seed_matrix_df = pd.read_csv("datasets/our_data/team_summary_data/seeds_matrix", index_col=0)

In [12]:
# get data into correct format
predictor_names = ["min_index_id", "max_index_id", "markov", "dominance", "rpi", "bad_losses", "tough_wins", "close_wins", "close_wins_perc", "weighted_wins", "past_resul", "momentum"] 
predictor_dfs = [markov_data, dominance, rpi, bad_losses, tough_wins, close_wins, close_wins_perc, weighted_wins, past_resul, momentum]                                           

In [13]:
pred, resp = mmg.generate_multiple_years_of_games(range(1987, 2001), 
                                                  seeds_arr, 
                                                  slots_arr, 
                                                  games_arr, 
                                                  ["min_index_id", "max_index_id", "markov", "dominance", "rpi", "bad_losses", "tough_wins", "close_wins", "close_wins_perc", "weighted_wins", "past_resul", "momentum"], 
                                                  [markov_data, dominance, rpi, bad_losses, tough_wins, close_wins, close_wins_perc, weighted_wins, past_resul, momentum],
                                                  scoring_dif = False)


In [29]:
scaler = StandardScaler().fit(pred.iloc[:, 2:])

log_reg_model = LogReg(C = 10)
log_reg_model.fit(scaler.transform(pred.iloc[:, 2:]), resp.values.T[0])

LogisticRegression(C=10, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

#### Test on 2002

In [43]:
reload(mmc)
reload(mmm)

<module 'march_madness_models' from 'march_madness_models.pyc'>

In [50]:
year = 2005
seeds = seeds_arr[year - 1985]
slots = slots_arr[year - 1985]
resul = games_arr[year - 1985]

# biased model ---> predicts upsets with proability p + .05 (where p comes from log reg model)
head_to_head_biased_model = mmm.ModelPredictor(log_reg_model, 
                                               scaler,
                                               predictor_dfs, 
                                               year, 
                                               simulation=False, 
                                               higher_seed_bias=True, 
                                               seeds_df=seeds, 
                                               higher_seed_bias_delta=.05)


# biased model with cooling ----> predicts upsets with proability p + .05 * cooling_factor (where p comes from log reg model)
head_to_head_biased_model_cooling = mmm.ModelPredictor(log_reg_model, 
                                                       scaler, 
                                                       predictor_dfs, 
                                                       year, 
                                                       seeds_df=seeds, 
                                                       simulation=False, 
                                                       higher_seed_bias=True, 
                                                       higher_seed_bias_delta=.01,
                                                       cooling = {6:10, 5:10, 4:10, 3:10, 2:-5, 1:-5}
                                                      )

# unbiased model ----> predicts based on head to head log reg model
head_to_head_unbiased_model = mmm.ModelPredictor(log_reg_model, scaler, predictor_dfs, year, simulation=False, higher_seed_bias=False, seeds_df=seeds)


# for comparison
tourney_actual = mmc.Tournament(seeds, slots, mmm.ActualTournament(resul))
tourney_top_seed = mmc.Tournament(seeds, slots, mmm.BasicPredictor())

# predict tournament
tourney_biased_model = mmc.Tournament(seeds, slots, head_to_head_biased_model)
tourney_biased_model_cooling = mmc.Tournament(seeds, slots, head_to_head_biased_model_cooling)
tourney_unbiased_model = mmc.Tournament(seeds, slots, head_to_head_unbiased_model)

In [51]:
tourney_biased_model.compare_to_dif_tournament(tourney_actual, tourney_top_seed, print_res=True)

Number Correct Our Model     : 10, Number Correct Dif Model : 3
R1: Number Correct Our Model : 3, Number Correct Dif Model : 2
R2: Number Correct Our Model : 2, Number Correct Dif Model : 0
R3: Number Correct Our Model : 1, Number Correct Dif Model : 1
R4: Number Correct Our Model : 1, Number Correct Dif Model : 0
R5: Number Correct Our Model : 2, Number Correct Dif Model : 0
R6: Number Correct Our Model : 1, Number Correct Dif Model : 0


(15, 10, 3)